In [1]:
!pip install -q PyMuPDF requests python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 41.4 MB/s eta 0:00:00


In [7]:
# FM Contract Assistant MVP - ContractCortex
import os
import json
import fitz  # PyMuPDF
import requests
from IPython.display import display, HTML
import ipywidgets as widgets
from google.colab import files  # For file upload in Colab

# Configuration
OPENROUTER_API_KEY = "your api"  # Replace with your actual key
MODEL = "openai/gpt-4o-mini"  # Or another supported model

# ----------------------------
# 1. Enhanced Contract Digitization with AI
# ----------------------------
def parse_contract_with_ai(pdf_text):
    """Use OpenRouter AI for contract parsing"""
    system_prompt = """You are a Facility Management contract specialist. Extract these key fields from the contract:
    - Parties (client/supplier)
    - Start/end dates
    - Maintenance types (PPM, reactive, projects)
    - SLAs (response times, KPIs)
    - Pricing (hourly rates, emergency charges)
    - Invoicing rules
    - Scope of work
    Return JSON format only. Highlight FM-specific terms. Do not add any commentary or markdown. Just return a JSON object.
    """

    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://fm-contract-assistant.local",
            "X-Title": "FM Contract Assistant"
        },
        data=json.dumps({
            "model": MODEL,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": pdf_text[:15000]}
            ],
            "temperature": 0.1,
            "max_tokens": 1000
        })
    )

    if response.status_code == 200:
        try:
            ai_response = response.json()['choices'][0]['message']['content']
            print("\n🔍 Raw AI Response:\n", ai_response[:500])  # Show the first 500 chars

            # Try to locate and extract JSON object from messy responses
            json_start = ai_response.find('{')
            json_end = ai_response.rfind('}') + 1
            cleaned_json = ai_response[json_start:json_end]

            return json.loads(cleaned_json)
        except Exception as e:
            return {"error": f"Failed to parse AI response: {str(e)}"}
    else:
        return {"error": f"API Error: {response.text}"}


def parse_contract_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = " ".join([page.get_text() for page in doc])
    return text, parse_contract_with_ai(text)

# ----------------------------
# 2. Data Verification Interface
# ----------------------------
class VerificationUI:
    def __init__(self, pdf_text, extracted_data):
        self.pdf_text = pdf_text
        self.data = extracted_data

        self.pdf_view = widgets.Textarea(
            value=pdf_text[:2000] + "... [TRUNCATED] ...",
            layout={'width': '800px', 'height': '400px'}
        )
        self.json_editor = widgets.Textarea(
            value=json.dumps(extracted_data, indent=2),
            layout={'width': '800px', 'height': '400px'}
        )
        self.save_btn = widgets.Button(
            description="Verify & Lock Data",
            button_style='success'
        )
        self.save_btn.on_click(self.on_save)

        display(widgets.HBox([self.pdf_view, self.json_editor]))
        display(self.save_btn)

    def on_save(self, btn):
        try:
            self.data = json.loads(self.json_editor.value)
            print("✅ Contract terms verified and locked!")
        except json.JSONDecodeError:
            print("❌ Invalid JSON format")

# ----------------------------
# 3. AI-Powered Training Quest Generator
# ----------------------------
def generate_training_quest(verified_data):
    user_prompt = f"""
    Create a facility management training module based on this contract data:
    {json.dumps(verified_data, indent=2)}

    Include:
    1. 3 multiple-choice questions about SLAs
    2. 2 scenario-based questions about scope of work
    3. 1 practical exercise for invoicing procedures
    Format as JSON with 'title' and 'questions' array.
    """

    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json"
        },
        data=json.dumps({
            "model": MODEL,
            "messages": [
                {"role": "user", "content": user_prompt}
            ],
            "temperature": 0.3,
            "response_format": {"type": "json_object"}
        })
    )

    return json.loads(response.json()['choices'][0]['message']['content'])

# ----------------------------
# 4. Intelligent Chatbot with Context
# ----------------------------
class ContractChatbot:
    def __init__(self, verified_data):
        self.context = f"""
        Facility Management Contract Knowledge Base:
        {json.dumps(verified_data, indent=2)}
        """

    def ask_question(self, question):
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {OPENROUTER_API_KEY}",
                "Content-Type": "application/json"
            },
            data=json.dumps({
                "model": MODEL,
                "messages": [
                    {
                        "role": "system",
                        "content": f"You are a FM contract expert. Answer strictly based on: {self.context}"
                    },
                    {"role": "user", "content": question}
                ],
                "temperature": 0.1
            })
        )
        return response.json()['choices'][0]['message']['content']

# ----------------------------
# Demo Workflow in Colab
# ----------------------------

# 1. Upload sample FM contract PDF
uploaded = files.upload()
pdf_filename = next(iter(uploaded))  # Get the uploaded file name

# 2. Process contract
pdf_text, ai_data = parse_contract_pdf(pdf_filename)
print("📄 AI Extracted Data:", json.dumps(ai_data, indent=2))

# 3. Verify data
verification_ui = VerificationUI(pdf_text, ai_data)

# 4. Generate training module
verified_data = ai_data  # Normally: verification_ui.data after user review
training = generate_training_quest(verified_data)
print("\n📚 AI-Generated Training:", json.dumps(training, indent=2))

# 5. Chatbot interaction
chatbot = ContractChatbot(verified_data)
print("\n🤖 Chat Session:")
print("Q: What's the emergency response SLA?")
print("A:", chatbot.ask_question("What's the emergency response SLA?"))
print("\nQ: How to process invoices?")
print("A:", chatbot.ask_question("How to process invoices?"))


Saving FM Contract.pdf to FM Contract (2).pdf

🔍 Raw AI Response:
 {
  "Parties": {
    "Client": "SG21",
    "Supplier": "N/A"
  },
  "StartEndDates": {
    "StartDate": "2023-08-22",
    "EndDate": "Spring 2024"
  },
  "MaintenanceTypes": {
    "PPM": "N/A",
    "Reactive": "N/A",
    "Projects": "Contracts MVP"
  },
  "SLAs": {
    "ResponseTimes": "N/A",
    "KPIs": "N/A"
  },
  "Pricing": {
    "HourlyRates": "N/A",
    "EmergencyCharges": "N/A"
  },
  "InvoicingRules": "N/A",
  "ScopeOfWork": "Develop consensus on a minimal viable product (MVP) for a Cont
📄 AI Extracted Data: {
  "Parties": {
    "Client": "SG21",
    "Supplier": "N/A"
  },
  "StartEndDates": {
    "StartDate": "2023-08-22",
    "EndDate": "Spring 2024"
  },
  "MaintenanceTypes": {
    "PPM": "N/A",
    "Reactive": "N/A",
    "Projects": "Contracts MVP"
  },
  "SLAs": {
    "ResponseTimes": "N/A",
    "KPIs": "N/A"
  },
  "Pricing": {
    "HourlyRates": "N/A",
    "EmergencyCharges": "N/A"
  },
  "InvoicingRules"

Button(button_style='success', description='Verify & Lock Data', style=ButtonStyle())


📚 AI-Generated Training: {
  "title": "Facility Management Training Module",
  "questions": [
    {
      "type": "multiple-choice",
      "question": "What does SLA stand for in facility management?",
      "options": [
        "A) Service Level Agreement",
        "B) Service Level Assessment",
        "C) Standard Level Agreement",
        "D) Service Level Adjustment"
      ],
      "correctAnswer": "A"
    },
    {
      "type": "multiple-choice",
      "question": "Which of the following is NOT typically included in an SLA?",
      "options": [
        "A) Response Times",
        "B) Maintenance Types",
        "C) Key Performance Indicators (KPIs)",
        "D) Pricing Structure"
      ],
      "correctAnswer": "D"
    },
    {
      "type": "multiple-choice",
      "question": "Why are SLAs important in facility management?",
      "options": [
        "A) They determine the hourly rates for services.",
        "B) They establish clear expectations between parties.",
        